In [1]:
setwd('/lustre/scratch117/cellgen/team297/kt16/Ziad/raw/Trans')

In [2]:
library(SoupX)
library(Seurat)
library(Matrix)
library(reticulate)
scp = import("scanpy")
sp = import("scipy.sparse")

In [3]:
# run soupx on transcriptome data
soupx <- function(dataDirs){
    print(dataDirs)
    # generate clusters
    dat <- Read10X(paste0(dataDirs, '/filtered_feature_bc_matrix/'))
    raw <- Read10X(paste0(dataDirs, '/raw_feature_bc_matrix/'))
    seu <- CreateSeuratObject(counts = dat$`Gene Expression`)
    seu[["percent_mt"]] <- PercentageFeatureSet(seu, pattern = "^MT-")
    seu <- NormalizeData(seu, verbose = FALSE)
    seu <- FindVariableFeatures(seu, verbose = FALSE)
    seu <- ScaleData(seu, vars.to.regress = c("percent_mt", 'nCount_RNA'), verbose = FALSE)
    seu <- RunPCA(seu, features = VariableFeatures(object = seu), verbose = FALSE)
    seu <- FindNeighbors(seu, verbose = FALSE)
    seu <- FindClusters(seu, algorithm = 4, verbose = FALSE)
    clusters <- Idents(seu)
    # run soupx
    sc <- load10X(dataDirs)	
    sc <- setClusters(sc, clusters)
    sc = autoEstCont(sc, doPlot = FALSE)
    out = adjustCounts(sc)
    colnames(out) <- paste0(dataDirs, '_', colnames(out))
    # correct ADT data too
    sc2 <- SoupChannel(raw$`Antibody Capture`, dat$`Antibody Capture`)
    sc2 <- setClusters(sc2, clusters)
    useToEst = estimateNonExpressingCells(sc2, nonExpressedGeneList = list(c('CD25+prot', 'PD1_prot')))
    sc2 = calculateContaminationFraction(sc2, list(c('CD25+prot','PD1_prot')), useToEst, forceAccept = TRUE)
    out2 = adjustCounts(sc2)    
    # combine the new matrices
    newx = sp$hstack(c(t(out), t(out2)), format = 'csr')
    adata = scp$read_10x_mtx(paste0(dataDirs, '/filtered_feature_bc_matrix'), gex_only = FALSE)
    adata$X = newx
    # save as a .h5ad file
    adata$write(paste0(dataDirs, '/adata_soupx_trans_cite.h5ad'), compression = 'gzip')

    # also create a version where only the soupx trans is appended to the original cite counts.
    newx = sp$hstack(c(t(out), t(dat$`Antibody Capture`)), format = 'csr')
    adata = scp$read_10x_mtx(paste0(dataDirs, '/filtered_feature_bc_matrix'), gex_only = FALSE)
    adata$X = newx
    # save as a .h5ad file
    adata$write(paste0(dataDirs, '/adata_soupx_trans.h5ad'), compression = 'gzip')
    
    # and a version using Mike's method
    # read in corrected ADT data
    cite <- read.delim(paste0(dataDirs, '/ADT_', dataDirs, '_bgCPM.txt.gz'), check.names = FALSE)
    cite <- tibble::column_to_rownames(cite, var = "ADT")
    cite <- Matrix::Matrix(as.matrix(cite), sparse = TRUE)
    # combine the new matrices
    newx = sp$hstack(c(t(out), t(cite)), format = 'csr')
    adata = scp$read_10x_mtx(paste0(dataDirs, '/filtered_feature_bc_matrix'), gex_only = FALSE)
    adata$X = newx
    # save as a .h5ad file
    adata$write(paste0(dataDirs, '/adata_soupx_trans_cite_bgshift.h5ad'), compression = 'gzip')
}

In [4]:
files = paste0('Sample_Fq', 1:32)
files <- files[-15]

In [5]:
for (i in seq_along(files)){
        soupx(files[i])
}

[1] "Sample_Fq1"


10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading raw count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading cell-only count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading extra analysis data where available

1170 genes passed tf-idf cut-off and 480 soup quantile filter.  Taking the top 100.

Using 1576 independent estimates of rho.

Estimated global rho of 0.01

Expanding counts from 21 clusters to 11651 cells.

Extremely high contamination estimated (0.72).  This likely represents a failure in estimating the contamination fraction.  Set forceAccept=TRUE to proceed with this value.

Estimated global contamination fraction of 72.33%

Expanding counts from 21 clusters to 11651 cells.



[1] "Sample_Fq2"


10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading raw count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading cell-only count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading extra analysis data where available

2331 genes passed tf-idf cut-off and 935 soup quantile filter.  Taking the top 100.

Using 1817 independent estimates of rho.

Estimated global rho of 0.01

Expanding counts from 22 clusters to 6908 cells.

Extremely high contamination estimated (0.73).  This likely represents a failure in estimating the contamination fraction.  Set forceAccept=TRUE to proceed with this value.

Estimated global contamination fraction of 73.40%

Expanding counts from 22 clusters to 6908 cells.



[1] "Sample_Fq3"


...

10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading raw count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading cell-only count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading extra analysis data where available

359 genes passed tf-idf cut-off and 238 soup quantile filter.  Taking the top 100.

Using 972 independent estimates of rho.

Estimated global rho of 0.01

Expanding counts from 14 clusters to 14159 cells.

Extremely high contamination estimated (0.81).  This likely represents a failure in estimating the contamination fraction.  Set forceAccept=TRUE to proceed with this value.

Estimated global contamination fraction of 81.44%

Expanding counts from 14 clusters to 14159 cells.



[1] "Sample_Fq31"


10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading raw count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading cell-only count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading extra analysis data where available

527 genes passed tf-idf cut-off and 328 soup quantile filter.  Taking the top 100.

Using 1131 independent estimates of rho.

Estimated global rho of 0.01

Expanding counts from 14 clusters to 3385 cells.

Extremely high contamination estimated (0.61).  This likely represents a failure in estimating the contamination fraction.  Set forceAccept=TRUE to proceed with this value.

Estimated global contamination fraction of 61.16%

Expanding counts from 14 clusters to 3385 cells.



[1] "Sample_Fq32"


10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading raw count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading cell-only count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading extra analysis data where available

402 genes passed tf-idf cut-off and 209 soup quantile filter.  Taking the top 100.

Using 964 independent estimates of rho.

Estimated global rho of 0.01

Expanding counts from 15 clusters to 7610 cells.

Extremely high contamination estimated (0.74).  This likely represents a failure in estimating the contamination fraction.  Set forceAccept=TRUE to proceed with this value.

Estimated global contamination fraction of 74.42%

Expanding counts from 15 clusters to 7610 cells.



In [4]:
file = 'Sample_Fq15'

In [6]:
soupx(file)

[1] "Sample_Fq15"


10X data contains more than one type and is being returned as a list containing matrices of each type.

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading raw count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading cell-only count data

10X data contains more than one type and is being returned as a list containing matrices of each type.

Loading extra analysis data where available

318 genes passed tf-idf cut-off and 185 soup quantile filter.  Taking the top 100.

Using 1225 independent estimates of rho.

Estimated global rho of 0.01

Expanding counts from 18 clusters to 10859 cells.

Extremely high contamination estimated (0.76).  This likely represents a failure in estimating the contamination fraction.  Set forceAccept=TRUE to proceed with this value.

Estimated global contamination fraction of 76.02%

Expanding counts from 18 clusters to 10859 cells.

